In [1]:
import geopandas as gpd


# Define grid resolution
resolution = 50

# Setup grid parameters with spatial origin and grid size
x0, y0 = 587000, 2350004
x1, y1 = 625000, 2368000

# Coordinates for the specific point
x, y = 601900.8911782742, 2356315.456879479

# Create a GeoDataFrame with the point geometry using the provided EPSG code from grid_params
gdf_boundary_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy([x], [y]), crs=6634)

# Grid configuration dictionary
fixed_parameters = {
    "path_to_dem_tif": "inputs/oahu_dem_coarsen.tif",
    "dem_tif_var_name": "elevtn",
    "path_to_rgh_tif": "inputs/sfincs.man",
    "path_to_outflow_shp": "inputs/polygon_isobat20.shp",
    "path_to_waterlevel_shp": "inputs/offshore_coastline.shp",
    "x0": x0,  # Origin location of the cell edge (x0)
    "y0": y0,  # Origin location of the cell edge (y0)
    "dx": resolution,  # Grid size in x-direction
    "dy": resolution,  # Grid size in y-direction
    "mmax": (x1 - x0) // resolution,  # Number of grid cells in x-direction
    "nmax": (y1 - y0) // resolution,  # Number of grid cells in y-direction
    "rotation": 0,  # Rotation angle in degrees (anti-clockwise from east)
    "epsg": 6634,  # Coordinate reference system (CRS) EPSG code
    "path_to_mask": "inputs/mask_compound_pol.shp",
    "nr_subgrid_pixels": 1,
    "gdf_boundary_points": gdf_boundary_points,
}

In [ ]:
import pandas as pd


waterlevels_forcing = pd.read_csv("inputs/df_twl_ig_recons.csv")
waterlevels_forcing

,time,Msetup,ig,WL,TWL_proxy
0,1941-01-01 00:00:00,0.161729,0.219623,-0.145173,0.126368
1,1941-01-01 03:00:00,0.121542,0.259632,-0.023173,0.228185
2,1941-01-01 06:00:00,0.137637,0.235158,-0.084174,0.171042
3,1941-01-01 09:00:00,0.171875,0.190463,-0.176174,0.090933
4,1941-01-01 12:00:00,0.104281,0.259614,0.037825,0.271913
...,...,...,...,...,...
208177,2022-12-31 09:00:00,0.123053,0.381927,0.157866,0.471883
208178,2022-12-31 12:00:00,0.120767,0.371098,0.153866,0.460182
208179,2022-12-31 15:00:00,0.180541,0.347042,-0.009135,0.344928
208180,2022-12-31 18:00:00,0.182940,0.339103,-0.026135,0.326356


In [3]:
metamodel_parameters = {
    "waterlevel_forcing": [
        waterlevels_forcing.iloc[i : i + 100][["TWL_proxy"]].rename(
            columns={"TWL_proxy": 0}
        )
        for i in range(0, len(waterlevels_forcing), 100)
    ]
}
metamodel_parameters

{'waterlevel_forcing': [           0
  0   0.126368
  1   0.228185
  2   0.171042
  3   0.090933
  4   0.271913
  ..       ...
  95  0.139030
  96  0.269646
  97  0.245951
  98  0.067617
  99  0.166755
  
  [100 rows x 1 columns],
              0
  100  0.520327
  101  0.574369
  102  0.293709
  103  0.118274
  104  0.246720
  ..        ...
  195  0.380525
  196  0.688093
  197  0.481122
  198  0.257417
  199  0.271002
  
  [100 rows x 1 columns],
              0
  200  0.239855
  201  0.189967
  202  0.084147
  203  0.287597
  204  0.684232
  ..        ...
  295  0.307538
  296  0.230818
  297  0.179837
  298  0.307917
  299  0.493708
  
  [100 rows x 1 columns],
              0
  300  0.516064
  301  0.344910
  302  0.251147
  303  0.317789
  304  0.267377
  ..        ...
  395  0.529300
  396  0.415910
  397  0.326355
  398  0.371987
  399  0.367146
  
  [100 rows x 1 columns],
              0
  400  0.235988
  401  0.203900
  402  0.394508
  403  0.526283
  404  0.483748
  ..      

In [4]:
from bluemath_tk.wrappers.sfincs.sfincs_wrapper import SfincsAlbaModelWrapper


sfincs_model = SfincsAlbaModelWrapper(
    templates_dir="templates",
    metamodel_parameters=metamodel_parameters,
    fixed_parameters=fixed_parameters,
    output_dir="Alba_CASES",
)

2025-05-05 18:17:28,781 - SfincsAlbaModelWrapper - WARNING - Parameter waterlevel_forcing is not in the default_parameters


In [ ]:
sfincs_model.build_cases()